## Project 1: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [1]:
import torch
# the number of entries in our database
num_entries = 5000
db = torch.rand(num_entries) > 0.5

In [2]:
def get_parallel_db(db, remove_index):
    """Returns a parallel database by removing
    one sample from a given index
    
    Args:
        db (tensor): The original database.
        remove_index (int): specifies which sample to remove.

    Returns:
        tensor: a pararllel database.
    """
    # TODO: Remove one sample from the database using the remove_index
    
    parallel_db = torch.cat((db[0:remove_index], 
                             db[remove_index+1:]))
    return parallel_db

get_parallel_db(db, 1)

tensor([1, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [3]:
def get_parallel_dbs(db):
    """Returns a list of all the possible parallel 
    databases of a given database
    
    Args:
        db (tensor): The original database.

    Returns:
        list: a list of pararllel databases.
    """
    parallel_dbs = list()
    
    # TODO: Create a list of parallel databases for every index
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
        
    return parallel_dbs

pdbs = get_parallel_dbs(db)
pdbs[:3]

[tensor([1, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8),
 tensor([1, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.uint8)]

In [4]:
def create_db_and_parallels(num_entries):
    """Initializes a databes and creates a list of 
    all the possible parallel databases of itself
    
    Args:
        num_entries (int): length of the original database.

    Returns:
        db (tensor): the original database.
        pdbs (list): a list of pararllel databases.
    """
    # TODO: Initialize a database and all its parallel databases
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

db, pdbs = create_db_and_parallels(20)
db

tensor([0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       dtype=torch.uint8)

# Project 2 - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [5]:
def query(db):
    """Queries the database by computing a mean"""
    return db.float().mean()

In [6]:
def sensitivity(query, n_entries=1000):
    """Computes the sensitivity of a given query
    for a particular database
    
    Args:
        query (fn): a function that queries the database.
        n_entries (int): size of the database.

    Returns:
        tensor: the resulting sensitivity.
    """
    # TODO: Create a custon database and its parallel databases
    db, pdbs = create_db_and_parallels(n_entries)
    
    # TODO: Query the full database
    full_db_result = query(db)
    
    # TODO: Loop through the parallel databases and compute the maximum sensitivity
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
            
    return sensitivity

In [7]:
sensitivity(query)

tensor(0.0005)

# Project 3: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [8]:
def query(db, threshold=5):
    """Computes the sum over a database and returns 
    whether that sum is greater than a certain threshold
    
    Args:
        db (tensor): randomly generated database.
        threshold (int): threshold to compare with query result.

    Returns:
        tensor: output of the query with threshold.
    """
    
    # TODO: Compute the sum over the database and check 
    # if it is graterer than the given threshold.
    
    return (db.sum() > threshold).float()

In [9]:
# TODO: create a database of size 10 and threshold of 5 and calculate the sensitivity of the function.
sensitivity(query, n_entries=10)

0

In [10]:
# TODO: re-initialize the database 10 times and calculate the sensitivity each time.
for i in range(10):
    sens_f = sensitivity(query, n_entries=10)
    print(sens_f)

0
0
0
tensor(1.)
0
0
0
0
0
tensor(1.)


# Project 4: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [11]:
# TODO: Construct a database with at least 10 rows
db, _ = create_db_and_parallels(100)

In [12]:
# TODO: Get a parallel db by removing the 10th row
pdb = get_parallel_db(db, remove_index=10)

In [13]:
# TODO: differencing attack using sum query
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [14]:
# TODO: differencing attack using mean query
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0043)

In [15]:
# TODO: differencing attack using threshold
(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [16]:
def query(db):
    """Adds random noise to the dataset using 
    local differential privacy and performs a
    query into the augmented database
    
    Args:
        db (tensor): randomly generated database.

    Returns:
        db_result (tensor): output of the query on the augmented database. 
        true_result (tensor): output of the query on the true database. 
    
    """
    # TODO: Perform a mean query on the original database
    true_result = torch.mean(db.float())
    
    # TODO: Simulate the coin flips (Tip: use the torch.rand method)
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    # TODO: Add the noise to the database using the simulated coin flips
    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
    # TODO: Perform a mean query on the augmented database
    # NOTE: You should remove the noise after computing the query
    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [17]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.1000)
Without Noise:tensor(0.3000)


In [18]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5200)
Without Noise:tensor(0.4900)


In [19]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5100)
Without Noise:tensor(0.4930)


In [20]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5076)
Without Noise:tensor(0.5007)
